In [1]:
import os
import shutil

original_db = "database/hotel_reservation_sample.db"
working_db = "database/hotel_reservation.db"
shutil.copy(original_db, working_db)

os.environ["DB"] = working_db

# Userstory 1
Als Gast möchte ich die verfügbaren Hotels durchsuchen, damit
ich dasjenige auswählen kann, welches meinen Wünschen
entspricht. Wünsche sind:

# Userstory 1.1
Ich möchte alle Hotels in einer Stadt durchsuchen,
damit ich das Hotel nach meinem bevorzugten Standort
(Stadt) auswählen kann.


In [3]:
import model
from business_logic.hotel_manager import HotelManager
from business_logic.address_manager import AddressManager

hotel_manager = HotelManager()

valid = False
while not valid:
    city = input("In welcher Stadt suchen Sie Ihr gewünschtes Hotel? ")
    hotels = hotel_manager.search_hotels_by_city(city)
    if hotels:
        print("Gefundene Hotels:")
        for hotel in hotels:
            print(f"{hotel.name} mit {hotel.stars} Sternen an der {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}")
        valid = True
    else:
        print("Keine Hotels gefunden")


Gefundene Hotels:
Hotel Baur au Lac mit 5 Sternen an der Bahnhofstrasse 1, Zürich, 8001


# Userstory 1.2
Ich möchte alle Hotels in einer Stadt nach der
Anzahl der Sterne (z.B. mindestens 4 Sterne) durchsuchen.


In [ ]:
import model
from business_logic.hotel_manager import HotelManager

hotel_manager = HotelManager()

valid = False
while not valid:
    city = input("In welcher Stadt suchen Sie Ihr gewünschtes Hotel? ")
    stars = int(input("Geben Sie die min. Anzahl Sterne an" ))
    hotels = hotel_manager.search_hotels_by_city_and_stars(city, stars)
    if hotels:
        print("Gefundene Hotels:")
        for hotel in hotels:
            print(f"{hotel.name} mit {hotel.stars} Sternen an der {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}")
        valid = True
    else:
        print("Keine Hotels gefunden")


# Userstory 1.3
Ich möchte alle Hotels in einer Stadt durchsuchen,
die Zimmer haben, die meiner Gästezahl entsprechen (nur 1
Zimmer pro Buchung).


In [8]:
from business_logic.hotel_manager import HotelManager

hotel_manager = HotelManager()

valid = False
while not valid:
    city = input("In welcher Stadt suchen Sie Ihr gewünschtes Hotel? ")
    try:
        guest_count = int(input("Geben Sie die min. Anzahl der Gäste an: "))
    except ValueError:
        print("Ungültige Eingabe, geben Sie eine Zahl an")

    hotels = hotel_manager.search_hotels_by_city_and_room_capacity(city, guest_count)

    if hotels:
        print("Gefundene Hotels:")
        for hotel in hotels:
            print(f"\n{hotel.name}, Sterne: {hotel.stars}"
                  f"\nAddresse: {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}"
                  f"\nZimmer: {hotel.room_number}, {hotel.description}, Kapazität: {hotel.max_guests}"
                  f"\nPreis pro Nacht: {hotel.price_per_night}")

        valid = True
    else:
        print("Keine Hotels gefunden")


Gefundene Hotels:

Hotel Baur au Lac, Sterne: 5
Addresse: Bahnhofstrasse 1, Zürich, 8001
Zimmer: 101, Single, Kapazität: 1
Preis pro Nacht: 250.0

Hotel Baur au Lac, Sterne: 5
Addresse: Bahnhofstrasse 1, Zürich, 8001
Zimmer: 102, Double, Kapazität: 2
Preis pro Nacht: 400.0


# Userstory 1.4
Ich möchte alle Hotels in einer Stadt durchsuchen,
die während meines Aufenthaltes ("von" (check_in_date)
und "bis" (check_out_date)) Zimmer zur Verfügung haben,
damit ich nur relevante Ergebnisse sehe.


In [5]:
from datetime import datetime
from business_logic.hotel_manager import HotelManager

hotel_manager = HotelManager()
valid = False
while not valid:
    city = input("In welcher Stadt suchen Sie Ihr gewünschtes Hotel? ")
    check_in_date = input("Geben Sie ihr gewünschtes Anreisedatum an (YYYY-MM-DD): ")
    check_out_date = input("Geben Sie ihr gewünschtes Abreisedatum an (YYYY-MM-DD): ")

    try:
        check_in_date = datetime.strptime(check_in_date, "%Y-%m-%d").date()
        check_out_date = datetime.strptime(check_out_date, "%Y-%m-%d").date()
    except ValueError:
        print("Ungültiges Datumsformat. Bitte unbedingt YYYY-MM-DD verwenden.\n")
        continue

    if check_in_date >= check_out_date:
        print("Fehler: Checkout muss nach dem Checkin liegen. Bitte erneut eingeben")
        continue

    hotels = hotel_manager.search_hotels_by_city_and_availability(city, check_in_date, check_out_date)

    if hotels:
        print("Gefundene Hotels:")
        for hotel in hotels:
            print(f"\n{hotel.name}, Sterne: {hotel.stars}"
                  f"\nAddresse: {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}"
                  f"\nZimmer: {hotel.room_number}, {hotel.description}, Kapazität: {hotel.max_guests}"
                  f"\nPreis pro Nacht: {hotel.price_per_night}"
                  f"\nAnreisedatum von bis: {check_in_date} - {check_out_date}"
            )
        valid = True
    else:
        print("Leider keine Verfügbarkeit in diesem Zeitraum. Versuchen Sie bitte einen anderen Zeitraum.")



Gefundene Hotels:

Hotel Baur au Lac, Sterne: 5
Addresse: Bahnhofstrasse 1, Zürich, 8001
Zimmer: 101, Single, Kapazität: 1
Preis pro Nacht: 250.0
Anreisedatum von bis: 2025-06-01 - 2025-06-05

Hotel Baur au Lac, Sterne: 5
Addresse: Bahnhofstrasse 1, Zürich, 8001
Zimmer: 102, Double, Kapazität: 2
Preis pro Nacht: 400.0
Anreisedatum von bis: 2025-06-01 - 2025-06-05


# Userstory 1.5
Ich möchte Wünsche kombinieren können, z.B. die
verfügbaren Zimmer zusammen mit meiner Gästezahl und der
mindest Anzahl Sterne

In [8]:
from datetime import datetime
from business_logic.hotel_manager import HotelManager

hotel_manager = HotelManager()
valid = False
while not valid:
    city = input("In welcher Stadt suchen Sie Ihr gewünschtes Hotel? ")
    try:
        check_in_str = input("Geben Sie ihr gewünschtes Anreisedatum an (YYYY-MM-DD): ")
        check_out_str = input("Geben Sie ihr gewünschtes Abreisedatum an (YYYY-MM-DD): ")
        check_in_date = datetime.strptime(check_in_str, "%Y-%m-%d").date()
        check_out_date = datetime.strptime(check_out_str, "%Y-%m-%d").date()
    except ValueError:
        print("Ungültiges Datumsformat. Bitte unbedingt YYYY-MM-DD verwenden.\n")
        continue

    if check_in_date >= check_out_date:
        print("Fehler: Checkout muss nach dem Checkin liegen. Bitte erneut eingeben")
        continue

    try:
        stars = int(input("Geben Sie die min. Anzahl Sterne an" ))
    except ValueError:
        print("Ungültige Eingabe, geben Sie eine Zahl an")
        continue

    try:
        guest_count = int(input("Geben Sie die min. Anzahl der Gäste an: "))
    except ValueError:
        print("Ungültige Eingabe, geben Sie eine Zahl an")
        continue


    hotels = hotel_manager.search_hotels_by_city_availability_stars_capacity(
        city,
        check_in_date,
        check_out_date,
        stars,
        guest_count
    )

    if hotels:
        print("Gefundene Hotels:")
        for hotel in hotels:
            print(f"\n{hotel.name}, Sterne: {hotel.stars}"
                  f"\nAddresse: {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}"
                  f"\nZimmer: {hotel.room_number}, {hotel.description}, Kapazität: {hotel.max_guests}"
                  f"\nPreis pro Nacht: {hotel.price_per_night}"
                  f"\nAnreisedatum von bis: {check_in_date} - {check_out_date}"
            )
        valid = True
    else:
        print("Leider keine Verfügbarkeit in diesem Zeitraum. Versuchen Sie bitte einen anderen Zeitraum.")



Gefundene Hotels:

Hotel Baur au Lac, Sterne: 5
Addresse: Bahnhofstrasse 1, Zürich, 8001
Zimmer: 101, Single, Kapazität: 1
Preis pro Nacht: 250.0
Anreisedatum von bis: 2025-06-01 - 2025-06-05

Hotel Baur au Lac, Sterne: 5
Addresse: Bahnhofstrasse 1, Zürich, 8001
Zimmer: 102, Double, Kapazität: 2
Preis pro Nacht: 400.0
Anreisedatum von bis: 2025-06-01 - 2025-06-05


# Userstory 1.6
Ich möchte die folgenden Informationen pro Hotel
sehen: Name, Adresse, Anzahl der Sterne.

In [12]:
from business_logic.hotel_manager import HotelManager

hotel_manager = HotelManager()

hotels = hotel_manager.get_all_hotel_info()
if not hotels:
    print("Keine Hotels gefunden.")
else:
    for hotel in hotels:
        print(f"\nHotel: {hotel.name}, Sterne: {hotel.stars}\n"
              f"Adresse: {hotel.address.street}, {hotel.address.city}, {hotel.address.zip_code}"
            )



Hotel: Hotel Baur au Lac, Sterne: 5
Adresse: Bahnhofstrasse 1, Zürich, 8001

Hotel: Four Seasons Hôtel des Bergues, Sterne: 5
Adresse: Rue du Rhône 42, Genève, 1204

Hotel: Grand Hotel National, Sterne: 5
Adresse: Pilatusstrasse 15, Luzern, 6003

Hotel: Bellevue Palace, Sterne: 5
Adresse: Marktgasse 59, Bern, 3011

Hotel: Les Trois Rois, Sterne: 5
Adresse: Freiestrasse 10, Basel, 4051


# Userstory 2
Als Gast möchte ich Details zu verschiedenen Zimmertypen
(Single, Double, Suite usw.), die in einem Hotel verfügbar
sind, sehen, einschliesslich der maximalen Anzahl von Gästen
für dieses Zimmer, Beschreibung, Preis und Ausstattung, um eine
fundierte Entscheidung zu treffen.
# Userstory 2.1
Ich möchte die folgenden Informationen pro Zimmer
sehen: Zimmertyp, max. Anzahl der Gäste, Beschreibung,
Ausstattung, Preis pro Nacht und Gesamtpreis.

In [7]:

from business_logic.room_manager import RoomManager

room_manager = RoomManager()

hotel_name_input = input("Geben Sie den exakten Hotelnamen ein: ").strip()
try:
    rooms = room_manager.get_room_info_by_hotel(hotel_name_input)
except Exception as e:
    print(f"Fehler bei der Abfrage: {e}")

if not rooms:
    print(f"Für Hotel „{hotel_name_input}“ wurden keine Zimmer gefunden.")
else:
    print(f"\nZimmer für „{hotel_name_input}“:\n")
    for r in rooms:
        print(
            f"  Zimmer-ID      : {r.room_id}\n"
            f"  Zimmernummer   : {r.room_number}\n"
            f"  Zimmertyp      : {r.room_type.description}\n"
            f"  Max. Gäste     : {r.room_type.max_guests}\n"
            f"  Preis/Nacht    : {r.price_per_night}\n")


Zimmer für „Hotel Baur au Lac“:

  Zimmer-ID      : 1
  Zimmernummer   : 101
  Zimmertyp      : Single
  Max. Gäste     : 1
  Preis/Nacht    : 250.0

  Zimmer-ID      : 2
  Zimmernummer   : 102
  Zimmertyp      : Double
  Max. Gäste     : 2
  Preis/Nacht    : 400.0



# Userstory 2.2
Ich möchte nur die verfügbaren Zimmer sehen, sofern
ich meinen Aufenthalt (von – bis) spezifiziert habe.


In [2]:
from datetime import datetime
from business_logic.room_manager import RoomManager  # oder: HotelManager

room_manager = RoomManager()


valid = False
while not valid:
    try:
        check_in_str = input("Geben Sie ihr gewünschtes Anreisedatum an (YYYY-MM-DD): ")
        check_out_str = input("Geben Sie ihr gewünschtes Abreisedatum an (YYYY-MM-DD): ")
        check_in_date = datetime.strptime(check_in_str, "%Y-%m-%d").date()
        check_out_date = datetime.strptime(check_out_str, "%Y-%m-%d").date()
    except ValueError:
        print("Ungültiges Datumsformat. Bitte unbedingt YYYY-MM-DD verwenden.\n")
        continue

    if check_out_date <= check_in_date:
        print("Fehler: Check-out muss nach Check-in liegen. Bitte erneut eingeben.\n")
        continue

    rooms = room_manager.get_available_rooms_by_date(check_in_date, check_out_date)
    valid = True

if not rooms:
    print(f"Im Zeitraum {check_in_date}–{check_out_date} wurden keine freien Zimmer gefunden.")
else:
    print(f"\nVerfügbare Zimmer für ({check_in_date} bis {check_out_date}):\n")
    for r in rooms:
        print(
            f"- Zimmer-ID       : {r.room_id}\n"
            f"  Zimmernummer    : {r.room_number}\n"
            f"  Zimmertyp       : {r.room_type.description}\n"
            f"  Max. Gäste      : {r.room_type.max_guests}\n"
            f"  Preis pro Nacht : {r.price_per_night}\n"
        )


Verfügbare Zimmer für (2025-01-01 bis 2025-01-05):

- Zimmer-ID       : 1
  Zimmernummer    : 101
  Zimmertyp       : Single
  Max. Gäste      : 1
  Preis pro Nacht : 250.0

- Zimmer-ID       : 2
  Zimmernummer    : 102
  Zimmertyp       : Double
  Max. Gäste      : 2
  Preis pro Nacht : 400.0

- Zimmer-ID       : 3
  Zimmernummer    : 201
  Zimmertyp       : Suite
  Max. Gäste      : 4
  Preis pro Nacht : 650.0

- Zimmer-ID       : 4
  Zimmernummer    : 301
  Zimmertyp       : Family Room
  Max. Gäste      : 5
  Preis pro Nacht : 900.0

- Zimmer-ID       : 5
  Zimmernummer    : 401
  Zimmertyp       : Penthouse
  Max. Gäste      : 6
  Preis pro Nacht : 1500.0



# Userstory 3
Als Admin des Buchungssystems möchte ich die Möglichkeit haben,
Hotelinformationen zu pflegen, um aktuelle Informationen im
System zu haben.
# Userstory 3.1
Ich möchte neue Hotels zum System hinzufügen

## Input Daten vom Admin

* Daten wie Strasse, Hotelname, Anzahl Sterne und Ort(Addresse)




In [2]:
# Userstory 3.1
from datetime import date
from business_logic.hotel_manager import HotelManager
from business_logic.address_manager import AddressManager
from model.hotel import Hotel
from model.address import Address

address_manager = AddressManager()
createded_address = address_manager.create_address("Teststrasse", "Zürich", 8001)
print(f"Neue Addresse: {createded_address.street}, {createded_address.city}, {createded_address.zip_code}")


hotel_manager = HotelManager()
h = Hotel(6, "Olympia", 3)
hotel = hotel_manager.create_hotel("Olympia", 3, createded_address)
print(f"Neues Hotel: {h.name}, Sterne: {h.stars}")

Neue Addresse: Teststrasse, Zürich, 8001
Neues Hotel: Olympia, Sterne: 3


# Userstory 3.2
Ich möchte Hotels aus dem System entfernen


In [3]:
from business_logic.hotel_manager import HotelManager

hotels = hotel_manager.get_all_hotel_info()

if not hotels:
    print("Keine Hotels in der Datenbank gefunden")
else:
    for hotel in hotels:
        print(f"{hotel.hotel_id}: {hotel.name}, {hotel.stars}")



while True:
    try:
        user_choice = int(input("Geben Sie die ID ein die Sie löschen möchten"))
    except ValueError:
        print("Ungültige Eingabe bitte geben sie eine Zahl ein")
    try:
        hotel_manager.delete_hotel(user_choice)
        print(f"Hotel mit ID: {user_choice} wurde erfolgreich gelöscht.\n")
        break
    except Exception as e:
        print(f"Fehler bei der Abfrage: {e}")





1: Hotel Baur au Lac, 5
2: Four Seasons Hôtel des Bergues, 5
3: Grand Hotel National, 5
4: Bellevue Palace, 5
5: ahah, 4
Hotel mit ID: 1 wurde erfolgreich gelöscht.



# Userstory 3.3
Ich möchte die Informationen bestimmter Hotels
aktualisieren, z. B. den Namen, die Sterne usw.


In [8]:
from business_logic.hotel_manager import HotelManager

hotel_manager = HotelManager()

hotels = hotel_manager.get_all_hotel_info()
if not hotels:
    print("Keine Hotels in der Datenbank vorhanden.")
else:
    for idx, hotel in enumerate(hotels, start=1):
        print(f"{idx} - {hotel.name}, {hotel.stars}")


while True:
    try:
        index = int(input("Geben Sie die ID ein, die Sie aktualisieren möchten: "))
    except ValueError:
        print("Ungültige Eingabe. Bitte eine ganze Zahl eingeben.\n")

    try:
        name = input("Geben Sie den neuen Hotelnamen ein: ").strip()
    except ValueError:
        print("Der Name muss ein String sein")
    if not name:
        print("Der Name darf nicht leer sein")
        continue

    try:
        stars = int(input("Geben Sie die neue Sternezahl (1–5) ein: "))
        if not 1 <= stars <= 5:
            print("Bitte eine Zahl zwischen 1 und 5 eingeben.\n")
            continue
    except ValueError:
        print("Ungültige Eingabe, bitte eine ganze Zahl eingeben.\n")
        continue
    break

try:
    hotel_manager.update_hotel(index, name, stars)
except Exception as e:
    print(f"Fehler beim Aktualisieren: {e}")

print("\nHotel wurde aktualisiert:")
print(f"- ID     : {index}")
print(f"- Name   : {name}")
print(f"- Sterne : {stars}\n")

for hotel in hotels:
    print(f"{hotel.hotel_id} - {hotel.name}, {hotel.stars}")

IndentationError: unexpected indent (2958857407.py, line 47)

# Userstory 4
Als Gast möchte ich ein Zimmer in einem bestimmten Hotel
buchen, um meinen Urlaub zu planen.


In [ ]:
from business_logic.booking_manager import BookingManager
from model.guest import Guest
from model.room import Room
from model.room_type import RoomType
from datetime import date


guest = Guest(1, "John", "Doe", "mail")
room = Room(1, 202, 80.90)
room_type = RoomType(1, "Standard", 1)
room.room_type = room_type

booking_manager = BookingManager()
b = booking_manager.create_booking(guest, room,1, date(2025, 1, 1), date(2025, 1, 3))
print(f" Checkin Date: {b.check_in_date}, Amount: {b.total_amount}")

# Userstory 8
Als Admin des Buchungssystems möchte ich alle Buchungen aller
Hotels sehen können, um eine Übersicht zu erhalten.

In [2]:
from business_logic.booking_manager import BookingManager

booking_manager = BookingManager()
bookings = booking_manager.get_all_bookings()

if bookings:
    print("Alle Buchungen:")
    for booking in bookings:
        print(f"Booking ID: {booking.bookingid}, Gast: {booking.guest.firstname} {booking.guest.lastname}, Hotel: {booking.room.hotel.name}, Zimmer: {booking.room.number}, Zeitraum: {booking.check_in_date} bis {booking.check_out_date}")
else:
    print("Keine Buchungen gefunden.")

AttributeError: 'BookingManager' object has no attribute 'get_all_bookings'

# Userstory 9
Als Admin möchte ich eine Liste der Zimmer mit ihrer
Ausstattung sehen, damit ich sie besser bewerben kann.


In [ ]:
from business_logic.room_manager import RoomManager

room_manager = RoomManager()
rooms = room_manager.get_all_rooms_with_equipment()

if rooms:
    print("Zimmer mit Ausstattung:")
    for room in rooms:
        equipment = ", ".join([e.name for e in room.equipment]) if room.equipment else "Keine Ausstattung"
        print(f"Zimmernummer: {room.number}, Hotel: {room.hotel.name}, Typ: {room.room_type.name}, Ausstattung: {equipment}")
else:
    print("Keine Zimmer gefunden.")